In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Standardeinstellungen und Konfigurationsoptionen für die Transpilierung
{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Paketversionen</b></summary>

Der Code auf dieser Seite wurde mit den folgenden Anforderungen entwickelt.
Wir empfehlen die Verwendung dieser oder neuerer Versionen.

```
qiskit[all]~=2.3.0
qiskit-ibm-runtime~=0.43.1
```
</details>
Abstrakte Schaltungen müssen transpiliert werden, da QPUs über eine begrenzte Anzahl von Basis-Gates verfügen und keine beliebigen Operationen ausführen können. Die Funktion des Transpilers besteht darin, beliebige Schaltungen so zu ändern, dass sie auf einer bestimmten QPU ausgeführt werden können. Dies geschieht durch Übersetzung der Schaltungen in die unterstützten Basis-Gates und durch Einführung von SWAP-Gates nach Bedarf, sodass die Konnektivität der Schaltung mit der der QPU übereinstimmt.

Wie in [Transpile with pass managers](transpile-with-pass-managers) erläutert, können Sie einen [Pass Manager](https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.PassManager) mit der Funktion [`generate_preset_pass_manager`](https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.generate_preset_pass_manager#qiskit.transpiler.generate_preset_pass_manager) erstellen und eine Schaltung oder Liste von Schaltungen an dessen [run](https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.PassManager#run)-Methode übergeben, um sie zu transpilieren. Sie können `generate_preset_pass_manager` aufrufen, indem Sie nur die Optimierungsstufe und das Backend übergeben und die Standardwerte für alle anderen Optionen verwenden, oder Sie können zusätzliche Argumente an die Funktion übergeben, um die Transpilierung feinabzustimmen.

## Grundlegende Verwendung ohne Parameter
In diesem Beispiel übergeben wir eine Schaltung und die Ziel-QPU an den Transpiler, ohne weitere Parameter anzugeben.

Erstellen Sie eine Schaltung und zeigen Sie das Ergebnis an:

In [1]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import grover_operator, DiagonalGate
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke

# Create circuit to test transpiler on
oracle = DiagonalGate([1] * 7 + [-1])
qc = QuantumCircuit(3)
qc.h([0, 1, 2])
qc = qc.compose(grover_operator(oracle))

# Add measurements to the circuit
qc.measure_all()

# View the circuit
qc.draw(output="mpl")

<Image src="../docs/images/guides/defaults-and-configuration-options/extracted-outputs/f7070db2-2b3e-4dcd-bbc7-cac7662867b3-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/defaults-and-configuration-options/extracted-outputs/f7070db2-2b3e-4dcd-bbc7-cac7662867b3-0.svg)

Transpilieren Sie die Schaltung und zeigen Sie das Ergebnis an:

In [2]:
from qiskit.transpiler import generate_preset_pass_manager

# Specify the QPU to target
backend = FakeSherbrooke()

# Transpile the circuit
pass_manager = generate_preset_pass_manager(
    optimization_level=1, backend=backend
)
transpiled_circ = pass_manager.run(qc)

# View the transpiled circuit
transpiled_circ.draw(output="mpl", idle_wires=False)

<Image src="../docs/images/guides/defaults-and-configuration-options/extracted-outputs/27ab746b-e3d7-49a7-b40b-d1e2d9ca6088-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/defaults-and-configuration-options/extracted-outputs/27ab746b-e3d7-49a7-b40b-d1e2d9ca6088-0.svg)

## Alle verfügbaren Parameter
Im Folgenden sind alle verfügbaren Parameter für die Funktion [`generate_preset_pass_manager`](https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.generate_preset_pass_manager#qiskit.transpiler.generate_preset_pass_manager) aufgeführt. Es gibt zwei Klassen von Argumenten: solche, die das Ziel der Kompilierung beschreiben, und solche, die beeinflussen, wie der Transpiler arbeitet.

Alle Parameter außer `optimization_level` sind optional. Vollständige Details finden Sie in der [Transpiler-API-Dokumentation](https://docs.quantum.ibm.com/api/qiskit/transpiler#transpiler-api).

- `optimization_level` (int) - Wie viel Optimierung an den Schaltungen durchgeführt werden soll. Ganzzahl im Bereich (0 - 3). Höhere Stufen erzeugen besser optimierte Schaltungen, auf Kosten einer längeren Transpilierungszeit. Weitere Details finden Sie unter [Set transpiler optimization level](set-optimization).

### Parameter zur Beschreibung des Kompilierungsziels:
Diese Argumente beschreiben die Ziel-QPU für die Schaltungsausführung, einschließlich Informationen wie der Coupling Map der QPU (die die Konnektivität der Qubits beschreibt), der von der QPU unterstützten Basis-Gates und der Fehlerraten der Gates.

Viele dieser Parameter werden ausführlich unter [Commonly used parameters for transpilation](common-parameters) beschrieben.

<details>
  <summary>
    **QPU (`Backend`) Parameter**
  </summary>

**Backend-Parameter** - Wenn Sie `backend` angeben, müssen Sie weder `target` noch andere Backend-Optionen angeben. Ebenso müssen Sie bei Angabe von `target` weder `backend` noch andere Backend-Optionen angeben.
- `backend` (Backend) - Wenn dies gesetzt ist, kompiliert der Transpiler die Eingabeschaltung für dieses Gerät. Wenn eine andere Option gesetzt ist, die diese Einstellungen beeinflusst, wie z. B. `coupling_map`, überschreibt sie die Einstellungen von `backend`.
- `target` (Target) - Ein Backend-Transpiler-Ziel. Normalerweise wird dies als Teil des Backend-Arguments angegeben, aber wenn Sie manuell ein Target-Objekt erstellt haben, können Sie es hier angeben. Dies überschreibt das Ziel von `backend`.
- `backend_properties` (BackendProperties) - Von einer QPU zurückgegebene Eigenschaften, einschließlich Informationen über Gate-Fehler, Auslesefehler, Qubit-Kohärenzzeiten usw. Finden Sie eine QPU, die diese Informationen bereitstellt, indem Sie `backend.properties()` ausführen.
- `timing_constraints` (Dict[str, int] | None) - Eine optionale Hardware-Beschränkung für die Zeitauflösung von Anweisungen. Diese Information wird durch die QPU-Konfiguration bereitgestellt. Wenn die QPU keine Beschränkung für die Zeitaufteilung von Anweisungen hat, ist `timing_constraints` `None` und es wird keine Anpassung vorgenommen. Eine QPU kann eine Reihe von Beschränkungen melden, nämlich:
    - `granularity`: Ein ganzzahliger Wert, der die minimale Puls-Gate-Auflösung in Einheiten von dt darstellt. Ein benutzerdefiniertes Puls-Gate sollte eine Dauer haben, die ein Vielfaches dieses Granularitätswerts ist.
    - `min_length`: Ein ganzzahliger Wert, der die minimale Puls-Gate-Länge in Einheiten von dt darstellt. Ein benutzerdefiniertes Puls-Gate sollte länger als diese Länge sein.
    - `pulse_alignment`: Ein ganzzahliger Wert, der eine Zeitauflösung der Startzeit von Gate-Anweisungen darstellt. Gate-Anweisungen sollten zu einem Zeitpunkt beginnen, der ein Vielfaches dieses Werts ist.
    - `acquire_alignment`: Ein ganzzahliger Wert, der eine Zeitauflösung der Startzeit von Messanweisungen darstellt. Messanweisungen sollten zu einem Zeitpunkt beginnen, der ein Vielfaches dieses Werts ist.
</details>

<details>
  <summary>
    **Layout- und Topologie-Parameter**
  </summary>

- `basis_gates` (List[str] | None) - Liste der Basis-Gate-Namen, zu denen aufgerollt werden soll. Zum Beispiel ['u1', 'u2', 'u3', 'cx']. Wenn `None`, nicht aufrollen.
- `coupling_map` (CouplingMap | List[List[int]]) - Gerichtete Coupling Map (möglicherweise benutzerdefiniert), die beim Mapping angesteuert werden soll. Wenn die Coupling Map symmetrisch ist, müssen beide Richtungen angegeben werden. Diese Formate werden unterstützt:
    - CouplingMap-Instanz
    - Liste - muss als Adjazenzmatrix angegeben werden, wobei jeder Eintrag alle gerichteten Zwei-Qubit-Interaktionen spezifiziert, die von der QPU unterstützt werden. Zum Beispiel: [[0, 1], [0, 3], [1, 2], [1, 5], [2, 5], [4, 1], [5, 3]]
- `inst_map` (List[InstructionScheduleMap] | None) - Zuordnung von Schaltungsoperationen zu Pulssequenzen. Wenn `None`, wird die `instruction_schedule_map` der QPU verwendet.
</details>

### Parameter zur Beeinflussung der Funktionsweise des Transpilers
Diese Parameter beeinflussen bestimmte Transpilierungsstufen. Einige von ihnen können mehrere Stufen beeinflussen, wurden aber der Einfachheit halber nur unter einer Stufe aufgeführt. Wenn Sie ein Argument angeben, wie z. B. `initial_layout` für die Qubits, die Sie verwenden möchten, überschreibt dieser Wert alle Passes, die ihn ändern könnten. Mit anderen Worten, der Transpiler ändert nichts, was Sie manuell angeben. Einzelheiten zu bestimmten Stufen finden Sie unter [Transpiler stages](transpiler-stages).

<details>
  <summary>
    **Initialisierungsstufe**
  </summary>

- `hls_config` (HLSConfig) - Eine optionale Konfigurationsklasse `HLSConfig`, die direkt an den `HighLevelSynthesis`-Transformationspass übergeben wird. Mit dieser Konfigurationsklasse können Sie die Listen der Synthese-Algorithmen und deren Parameter für verschiedene High-Level-Objekte angeben.
- `init_method` (str) - Der Plugin-Name, der für die Initialisierungsstufe verwendet werden soll. Standardmäßig wird kein externes Plugin verwendet. Sie können eine Liste der installierten Plugins anzeigen, indem Sie `list_stage_plugins()` mit `init` als Stage-Name-Argument ausführen.
- `unitary_synthesis_method` (str) - Der Name der zu verwendenden unitären Synthesemethode. Standardmäßig wird `default` verwendet. Sie können eine Liste der installierten Plugins anzeigen, indem Sie `unitary_synthesis_plugin_names()` ausführen.
- `unitary_synthesis_plugin_config` (dict) - Ein optionales Konfigurationswörterbuch, das direkt an das unitäre Synthese-Plugin übergeben wird. Standardmäßig hat diese Einstellung keine Auswirkung, da die standardmäßige unitäre Synthesemethode keine benutzerdefinierte Konfiguration annimmt. Die Anwendung einer benutzerdefinierten Konfiguration sollte nur notwendig sein, wenn ein unitäres Synthese-Plugin mit dem Argument `unitary_synthesis` angegeben wird. Da dies für jedes unitäre Synthese-Plugin spezifisch ist, lesen Sie bitte die Dokumentation des Plugins, um zu erfahren, wie Sie diese Option verwenden.
</details>

<details>
  <summary>
    **Layout-Stufe**
  </summary>

- `initial_layout` (Layout | Dict | List) - Anfangsposition der virtuellen Qubits auf physischen Qubits. Wenn dieses Layout die Schaltung mit den `coupling_map`-Beschränkungen kompatibel macht, wird es verwendet. Das endgültige Layout ist nicht garantiert gleich, da der Transpiler Qubits durch Swaps oder andere Mittel permutieren kann. Vollständige Details finden Sie im [Initial layout-Abschnitt.](common-parameters#initial-layout)
- `layout_method` (str) - Name des Layout-Auswahlpasses (`default`, `dense`, `sabre` und `trivial`). Dies kann auch der externe Plugin-Name sein, der für die Layout-Stufe verwendet werden soll. Sie können eine Liste der installierten Plugins anzeigen, indem Sie `list_stage_plugins()` mit `layout` als `stage_name`-Argument ausführen. Der Standardwert ist `sabre`.
</details>

<details>
  <summary>
    **Routing-Stufe**
  </summary>

- `routing_method` (str) - Name des Routing-Passes (`basic`, `lookahead`, `default`, `sabre` oder `none`). Dies kann auch der externe Plugin-Name sein, der für die Routing-Stufe verwendet werden soll. Sie können eine Liste der installierten Plugins anzeigen, indem Sie `list_stage_plugins()` mit `routing` als `stage_name`-Argument ausführen. Der Standardwert ist `sabre`.
</details>

<details>
  <summary>
    **Übersetzungsstufe**
  </summary>

- `translation_method` (str) - Name des Übersetzungspasses (`default`, `synthesis`, `translator`, `ibm_backend`, `ibm_dynamic_circuits`, `ibm_fractional`). Dies kann auch der externe Plugin-Name sein, der für die Übersetzungsstufe verwendet werden soll. Sie können eine Liste der installierten Plugins anzeigen, indem Sie `list_stage_plugins()` mit `translation` als `stage_name`-Argument ausführen. Der Standardwert ist `translator`.
</details>

<details>
  <summary>
    **Optimierungsstufe**
  </summary>

- `approximation_degree` (float, im Bereich 0-1 | None) - Heuristisches Maß für die Schaltungsapproximation (1.0 = keine Approximation, 0.0 = maximale Approximation). Der Standardwert ist 1.0. Die Angabe von `None` setzt den Approximationsgrad auf die gemeldete Fehlerrate. Weitere Details finden Sie im [Approximation degree-Abschnitt](common-parameters#approx-degree).
- `optimization_method` (str) - Der Plugin-Name, der für die Optimierungsstufe verwendet werden soll. Standardmäßig wird kein externes Plugin verwendet. Sie können eine Liste der installierten Plugins anzeigen, indem Sie `list_stage_plugins()` mit `optimization` als `stage_name`-Argument ausführen.
</details>

<details>
  <summary>
    **Scheduling-Stufe**
  </summary>

- `scheduling_method` (str) - Name des Scheduling-Passes. Dies kann auch der externe Plugin-Name sein, der für die Scheduling-Stufe verwendet werden soll. Sie können eine Liste der installierten Plugins anzeigen, indem Sie `list_stage_plugins()` mit `scheduling` als `stage_name`-Argument ausführen.
  * 'as_soon_as_possible': Plant Anweisungen gierig, so früh wie möglich auf einer Qubit-Ressource (Alias: `asap`).
  * 'as_late_as_possible': Plant Anweisungen spät, d.h. hält Qubits nach Möglichkeit im Grundzustand (Alias: `alap`). Dies ist die Standardeinstellung.
</details>

<details>
  <summary>
    **Transpiler-Ausführung**
  </summary>

- `seed_transpiler` (int) - Setzt Zufallszahlen-Seeds für die stochastischen Teile des Transpilers.
</details>

Die folgenden Standardwerte werden verwendet, wenn Sie keinen der oben genannten Parameter angeben. Weitere Informationen finden Sie auf der [API-Referenzseite](../api/qiskit/transpiler_preset) der Methode:

In [3]:
generate_preset_pass_manager(
    optimization_level=1,
    backend=None,
    target=None,
    basis_gates=None,
    coupling_map=None,
    initial_layout=None,
    layout_method=None,
    routing_method=None,
    translation_method=None,
    scheduling_method=None,
    approximation_degree=1.0,
    seed_transpiler=None,
    unitary_synthesis_method="default",
    unitary_synthesis_plugin_config=None,
    hls_config=None,
    init_method=None,
    optimization_method=None,
)

## Next steps

<Admonition type="tip" title="Recommendations">
    - Learn how to [Set the optimization level](set-optimization).
    - Review more [Commonly used parameters](common-parameters).
    - Learn how to [Set the optimization level when using Qiskit Runtime.](./runtime-options-overview)
    - Visit the [Transpile with pass managers](transpile-with-pass-managers) topic.
    - For examples, see [Representing quantum computers.](./represent-quantum-computers)
    - Learn [how to transpile circuits](/docs/guides/circuit-transpilation-settings) as part of the Qiskit patterns workflow using Qiskit Runtime.
    - Review the [Transpile API documentation](/docs/api/qiskit/transpiler).
</Admonition>